# Data Preprocessing Project

## Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

<div id="table"></div>

### Table of contents:

1. [Open the data file and have a look at the general information.](#point1) 
2. [Data preprocessing](#point2) <br>
    2.1. [Processing missing values](#point2_1) <br>
    2.2. [Data type replacement](#point2_2) <br>
    2.3. [Processing duplicates](#point2_3) <br>
    2.4. [Categorizing data](#point2_4) <br>
3. [Answer these questions:](#point3) <br>
    3.1. [Is there a relation between having kids and repaying a loan on time?](#point3_1) <br>
    3.2. [Is there a relation between marital status and repaying a loan on time?](#point3_2) <br>
    3.3. [Is there a relation between income level and repaying a loan on time?](#point3_3) <br>
    3.4. [How do different loan purposes affect on-time repayment of the loan?](#point3_4) <br>
4. [General conclusion](#point4)

<a id="point1"></a>
### Open the data file and have a look at the general information. 

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [3]:
#downloading data, changing column names, checking general info
data = pd.read_csv('/datasets/credit_scoring_eng.csv')
new_column_names = ['children', 'days_employed', 'age', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']
data.set_axis(new_column_names, axis='columns', inplace=True) 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
age                 21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


We import the libraries we will be working with. We change the names of the column 'dob_years' to 'age' so it's easier to understand what values are there. Data types look OK, exept for 'days_employed' column, we'll change it to 'int' later. We also see there are missing values in 'days_employed' and 'total_income' columns, a lot for 'retiree' income type, but also in another income types.

<a id="point2"></a>
### Data preprocessing

<a id="point2_1"></a>
#### Processing missing values

In [4]:
#replacing wrong numbers (-1 kids, 20 kids)
data['children'] = data['children'].replace({-1:1, 20:2})

In [5]:
#filling missing values, replacing too big values with mean
data['days_employed'] = data['days_employed'].abs()
data['days_employed'] = data['days_employed'].fillna(value=0) 
days_employed_real = data['days_employed'] <= 15660
days_employed_mean = data.loc[days_employed_real, 'days_employed'].mean()
data.loc[data['days_employed'] > 15660, 'days_employed'] = days_employed_mean

In [6]:
#creating new column "years_employed" from "days_employed"
data['years_employed'] = data['days_employed'] / 261 

In [7]:
#replacing 0 with mean age
age_mean = int(data['age'].mean())
data['age'] = data['age'].replace(0, age_mean)

In [8]:
#checking education values
data['education'] = data['education'].str.lower() 
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [9]:
#checking family status values
data['family_status'].value_counts()
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [10]:
#replacing values in "gender" column
data['gender'].value_counts() 
data = data.drop(data[data['gender'] == 'XNA'].index)

In [11]:
#checking and replacing income type values
data['income_type'].value_counts()
data.drop(data[data['income_type'].isin(['entrepreneur','unemployed','student','paternity / maternity leave'])].index, inplace=True)
data['income_type'].value_counts()

employee         11119
business          5084
retiree           3856
civil servant     1459
Name: income_type, dtype: int64

In [ ]:
#creating new column "annual income" based on grouping by education type
data['total_income'].value_counts()
annual_income_mean = data.groupby('education')['total_income'].mean()
data['annual_income'] = data.groupby('education')['total_income'].transform(lambda grp: grp.fillna(np.mean(grp)))

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21518 entries, 0 to 21524
Data columns (total 14 columns):
children            21518 non-null int64
days_employed       21518 non-null float64
age                 21518 non-null int64
education           21518 non-null object
education_id        21518 non-null int64
family_status       21518 non-null object
family_status_id    21518 non-null int64
gender              21518 non-null object
income_type         21518 non-null object
debt                21518 non-null int64
total_income        19345 non-null float64
purpose             21518 non-null object
years_employed      21518 non-null float64
annual_income       21518 non-null float64
dtypes: float64(4), int64(5), object(5)
memory usage: 2.5+ MB


Let's check all the columns one after another. In 'children' column we see range from 0 to 5 children, and then suddenly 47 families with 20 children, but no families with 6, 7, 10 or 15 - so we assume it's a typo and change it to 2. There also can't be negative number of children, so we change -1 to 1.

In the 'days_employed' column we have negative numbers, so we change it to positive. There are also values missing, so we fill them with 0. Also about 10% of the data is corrupted - the numbers are too big to be true, maybe there's a problem with a floating point. Meanwhile we'll replace corrupted data with mean value (mean value for the column without the exaggerated numbers). I wish I coulld investigate this column further but it's the best I can do for now. 

We add new column 'years_employed' by dividing 'days_employed' by mean number of working days in a year, but actually we need this column only so it's easier to understand then number of days.

In the 'age' column we change 0 to mean_value, since there cannot be customers 0 years old. 

In 'education' column we just need to change characters to low case, and then check that we have 5 categories, and also 5 categories in the 'education_id' column, and the numbers are identical. 

In 'family_status' and 'family_status_id' columns numbers are also identical so we leave it as it is.

In 'gender' column we have 1 row with "XNA" value, so we just drop the row. 

In the 'income_type' column we have few values that only appear in 1-2 rows, so we might just drop them without affecting the overall statistics. In the 'total_income' column we have missing values, we will fill them with mean annual income for each group based on education level so we made a new column 'annual_income' we will be working with.

The data looks much cleaner. Putting it in order took me a lot of time and effort, I managed to drop only 7 rows, so it won't affect the overall statistics. Unfortunately, I have no idea how to explain those artifacts in 'days_employed' column. Is it a problem with floating point? I wish I was more experienced so that I could investigate it properly.

<a id="point2_2"></a>
#### Data type replacement

In [13]:
columns=['days_employed', 'annual_income']
data[columns] = data[columns].astype('int')

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21518 entries, 0 to 21524
Data columns (total 14 columns):
children            21518 non-null int64
days_employed       21518 non-null int64
age                 21518 non-null int64
education           21518 non-null object
education_id        21518 non-null int64
family_status       21518 non-null object
family_status_id    21518 non-null int64
gender              21518 non-null object
income_type         21518 non-null object
debt                21518 non-null int64
total_income        19345 non-null float64
purpose             21518 non-null object
years_employed      21518 non-null float64
annual_income       21518 non-null int64
dtypes: float64(2), int64(7), object(5)
memory usage: 2.5+ MB


Now the data looks even prettier. I changed 'days_employed' data type since half of the day doesn't make sense and it's easier to work with integer numbers. Same for annual income, half of the dollar doesn't really make a difference.

<a id="point2_3"></a>
#### Processing duplicates

In [14]:
data.duplicated().sum()
data.drop_duplicates(inplace=True)

There were only 54 duplicates so we can just drop them. I used this method because it's the only one I know but I have to say it feels too easy so I might be wrong.

<a id="point2_4"></a>
#### Categorizing data

In [15]:
#creating differentiation based on annual income
data['annual_income'].describe()

def sort_annual_income(income):
    if income < desc['25%']:
        return 'poverty'
    if income < desc['50%']:
        return 'low'
    if income < desc['75%']:
        return 'middle'
    else:
        return 'high'
    
data['income_category'] = data['annual_income'].apply(sort_annual_income)
data['income_category'].value_counts()
data.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,annual_income,income_category
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,32.328249,40620,high
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,15.420704,17932,middle
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21.545680,23341,middle
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,15.803629,42820,high
4,0,2063,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,7.906792,25378,upper middle


In [42]:
#lemmatizing "purpose" column to define most popular purposes
wordnet_lemma = WordNetLemmatizer()

def lemmatize_text(text):
    words = nltk.word_tokenize(text)
    return [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]

data['lemmas'] = data['purpose'].apply(lemmatize_text)

data['lemmas'].value_counts()

words_list = []
for x in data['lemmas']:
    for w in x:
        words_list.append(w)
        
words_list_counted = Counter(words_list)
words_frequency = pd.DataFrame.from_records(words_list_counted.most_common(), columns=['word','count'])
print(words_frequency)

             word  count
0               a   5103
1            real   4462
2          estate   4462
3             car   4305
4        purchase   3306
5       education   3109
6              to   3064
7              of   2993
8     transaction   2604
9        property   2536
10             my   2390
11            buy   2357
12        wedding   2323
13            own   2237
14          house   1904
15        housing   1904
16         buying   1633
17     commercial   1311
18            for   1288
19            the   1284
20           with   1277
21       building   1243
22    second-hand    964
23     university    948
24  supplementary    906
25        getting    868
26       ceremony    791
27         having    767
28           have    765
29        renting    650
30            out    650
31         family    638
32   construction    634
33     renovation    606
34    residential    605
35          going    496
36            get    446
37             an    442
38        profile    436


In [49]:
#creating new column with purpose category

def purpose_category(element):
    if 'estate' in element or 'out' in element:
        element = 'real property'
    elif ('housing' in element or 'house' in element or 'construction' in element or 'property' in element) & ('renting' not in element):
        element = 'personal property'
    elif 'university' in element or 'education' in element or 'educated' in element:
        element = 'education'
    elif 'wedding' in element:
        element = 'wedding'
    elif 'car' in element:
        element = 'car'
    return element
    
data['purpose_category'] = data['lemmas'].apply(purpose_category)

data.groupby('income_category')['purpose_category'].value_counts()

income_category  purpose_category 
high             personal property    1440
                 real property        1292
                 car                  1092
                 education             963
                 wedding               585
low              personal property    1414
                 real property        1265
                 car                  1049
                 education            1047
                 wedding               604
middle           personal property    1302
                 real property        1243
                 car                  1077
                 education             968
                 wedding               547
upper middle     personal property    1538
                 real property        1312
                 car                  1087
                 education            1035
                 wedding               587
Name: purpose_category, dtype: int64

We make a new column with income category based on annual income - from low to high. We used the describe() method to group the data according to quartile.

I checked the lemmatized purposes with values_counts() method, but they are still not in the order.I made a nested loop to put all the lemmas in one list, so I could use a Counter. Then I turned Counter output to a new DataFrame, already sorted. Now we can see which categories we should create.

If there's 'car' in a list - then it goes to 'car' category, same with wedding. For 'education' category key words are 'education', 'educated' and 'university'. Property category for me divides in two: personal and commercial. For personal key words are 'house', 'housing', 'construction' or 'property' without 'renting' since we have 'property for renting out' purpose, which is commercial. Real property's key words are 'estate' and 'out', which I added for the 'buying property for renting out' purpose. I didn't use key word 'buying' because it also shows up in 'buying my own car' purpose.


<a id="point3"></a>
### Answer these questions:

<a id="point3_1"></a>
#### Is there a relation between having kids and repaying a loan on time?

In [17]:
#children_correlation = (data.groupby('children')['debt'].sum()) / (data.groupby('children')['debt'].count() * 0.01)
children_correlation = (data.groupby('children')['debt'].mean() * 100)
children_correlation

children
0    7.546500
1    9.147095
2    9.449929
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64

We see that default rate is lower (only 7.5%) for people without children, as for people with children - default rate is from 8.18% to 9.75%, so it doesn't really matter how many children you have - the chance that people with children won't repay loan on time is higher.

<a id="point3_2"></a>
#### Is there a relation between marital status and repaying a loan on time?

In [18]:
#family_status_correlation = (data.groupby('family_status')['debt'].sum()) / (data.groupby('family_status')['debt'].count() * 0.01)
family_status_correlation = data.groupby('family_status')['debt'].mean() * 100
family_status_correlation

family_status
civil partnership    9.353905
divorced             7.112971
married              7.530804
unmarried            9.754361
widow / widower      6.569343
Name: debt, dtype: float64

We see that for people who never were married (unmarried and civil partnership) default rate is 9.75% and 9.35% meanwhile for married and divorced it's almost the same - 7.53% and 7.11%, and for widows and widowers it's even lower - only 6.56%. We can say that probably people who get married are more financially stable so that's why we see these numbers.

<a id="point3_3"></a>
#### Is there a relation between income level and repaying a loan on time?

In [19]:
#income_correlation = (data.groupby('income_category')['debt'].sum()) / (data.groupby('income_category')['debt'].count() * 0.01)
income_correlation = data.pivot_table(index='income_category', values='debt') * 100
income_correlation

,debt
income_category,
high,6.831720
low,7.901097
middle,8.837843
upper middle,8.868502


We see that people with low income have default rate of 7.9%, people with middle income - 8.83%, people with upper middle - 8.86% and people with high income - 6.8%. We  see relation, the higher is your income - the higher is the chance you will repay loan on time. As for people with low income - I guess when you do not have a lot of money - you have to be more responsible with it, so default rate is lower than for middle-class.

<a id="point3_4"></a>
#### How do different loan purposes affect on-time repayment of the loan?

In [20]:
#purpose_correlation = (data.groupby('purpose_category')['debt'].sum()) / (data.groupby('purpose_category')['debt'].count() * 0.01)
purpose_correlation = data.pivot_table(index='purpose_category', values='debt') * 100
purpose_correlation

,debt
purpose_category,
car,9.337979
education,9.220035
personal property,6.919564
real property,7.570423
wedding,8.006888


We see that personal property has the lowest default rate - 6.91% because it's very important and people do find way to repay money on time so that they have a place to live. Real property purpose has pretty low default rate (7.57%) which makes sence since it it commercial purpose. Wedding has a 8% default rate - i guess newly married couple gets some money as presents so it's easier to return. As for getting a car or an education - default rates are pretty much the same - 9.33% and 9.22% - i guess that not 100% of students finish their education and can repay sucessfully and as for car purpose - people who need loan to buy a car don't usually have a lot of money, other way they would just buy it, so chance that they won't pay on time is pretty high. 

<a id="point4"></a>
### General conclusion

This is my first project ever of its kind. I am not very familiar with all these methods so I am sure I made a lot of mistakes and there are a lot of things I could do easier and better. I didn't really use 'income_type' column, i guess it can affect my conclusion. As for the numbers I got - I think it is pretty obvious that people without children have better chance to repay the loan on time, same with high income. 